<a href="https://colab.research.google.com/github/dvignoles/nyc-transpo/blob/master/nyc_transpo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from datetime import datetime
import numpy as np
import requests, zipfile, io, os, re, csv

TAXI_CSV_PATH = "./taxi_data"
BIKE_CSV_PATH = "./bike_data"

# We will use with 2013-2018 June data since June 2013 is when Citi Bike was started, and 2013-2018 data is available for both taxi and bikes.
# We will also filter down to the first week of the month, because the data is an absolute unit
YEARS = ["2013", "2014", "2015", "2016", "2017", "2018"]

In [0]:
# Set up directories and util functions to store the downloaded csv files

if os.path.isdir(TAXI_CSV_PATH) is False:
  os.mkdir(TAXI_CSV_PATH)
if os.path.isdir(BIKE_CSV_PATH) is False:
  os.mkdir(BIKE_CSV_PATH)

  
def save_csv(file_path, http_data):
    with open(file_path, "w") as f:
      writer = csv.writer(f)
      reader = csv.reader(http_data.text.splitlines())
      for row in reader:
        writer.writerow(row)
        
def get_csv_name(data_type, year, two_weeks_only=False):
  if data_type is "taxi":
    return TAXI_CSV_PATH + "/" + year + "-06" + ("_parsed" if two_weeks_only is True else "") +  ".csv"
  elif data_type is "bike":
    return BIKE_CSV_PATH + "/" + year + "-06" + ("_parsed" if two_weeks_only is True else "") + ".csv"
  else:
    raise Exception("Data type must be taxi or bike")

# Taxi Dataframes
Since these downloads take a long time, do NOT reset all run times when it asks you to! It will wipe all downloaded files, and you'll have to download them again!

In [0]:
# Download and save the files first so they don't have to be redownloaded every time

for n in YEARS:
  file_path = get_csv_name("taxi", n, False)
  if os.path.isfile(file_path) is False:
    data = requests.get("https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_" + n + "-06.csv")
    save_csv(file_path, data)

# Load the actual data

dtypes = {
    "vendor_id": "str",
    "VendorID": "str",
    "pickup_datetime": "str",
    "dropoff_datetime": "str",
    "tpep_pickup_datetime": "str",
    "tpep_dropoff_datetime": "str",
    "passenger_count": np.int32,
    "trip_distance": np.float32,
    "pickup_longitude": np.float32,
    "pickup_latitude": np.float32,
    "PULocationID": np.int32,
    "DOLocationID": np.int32,
    "rate_code": np.int32,
    "RateCodeID": np.int32,
    "store_and_fwd_flag": "str",
    "dropoff_longitude": np.float32,
    "dropoff_latitude": np.float32,
    "payment_type": "str",
    "fare_amount": np.float32,
    "surcharge": np.float32,
    "mta_tax": np.float32,
    "tip_amount": np.float32,
    "tolls_amount": np.float32,
    "total_amount": np.float32
}

def get_parsed_taxi_df(year):
  # See if the sorted data was saved before, if so use it
  sorted_file_path = get_csv_name("taxi", year, True)
  if os.path.isfile(sorted_file_path) is True:
    df_toreturn = pd.read_csv(sorted_file_path, dtype=dtypes)
    df_toreturn['pickup_datetime_parsed'] = pd.to_datetime(df_toreturn['pickup_datetime_parsed'])
    return df_toreturn
  
  # Otherwise, load the full file, sort it, and save it for future reference
  print("getting full data")
  df_temp = pd.read_csv(get_csv_name("taxi", year, False), dtype=dtypes)
  
  # Sort by date
  df_temp.sort_values(by=df_temp.columns[1], inplace=True, kind='mergesort')
  df_temp['pickup_datetime_parsed'] = pd.to_datetime(df_temp.iloc[:,1])
  
  # Get only the first week of the month
  df_temp = df_temp[(df_temp['pickup_datetime_parsed'] > (year + '-06-01')) & (df_temp['pickup_datetime_parsed'] < (year + '-06-08'))]
  df_temp.to_csv(sorted_file_path)
  return df_temp
  
# The util function MUST be used to save memory while loading in the entire dataframes for each year.
# IGNORE MEMORY LIMIT WARNINGS! The data is garbage collected every time the function exits for each year.

df_taxi_2013 = get_parsed_taxi_df("2013")
df_taxi_2014 = get_parsed_taxi_df("2014")
df_taxi_2015 = get_parsed_taxi_df("2015")
df_taxi_2016 = get_parsed_taxi_df("2016")
df_taxi_2017 = get_parsed_taxi_df("2017")
df_taxi_2018 = get_parsed_taxi_df("2018")

In [9]:
df_taxi_2016.tail()

,Unnamed: 0,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,...,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,pickup_datetime_parsed
2670194,3587069,1,2016-06-07 23:59:59,2016-06-08 00:33:09,1,5.60,-73.996986,40.737179,5,N,...,40.748734,1,45.0,0.0,0.0,9.05,10.50,0.3,64.849998,2016-06-07 23:59:59
2670195,3587070,1,2016-06-07 23:59:59,2016-06-08 00:06:48,1,1.10,-73.973160,40.763977,1,N,...,40.763103,2,6.5,0.5,0.5,0.00,0.00,0.3,7.800000,2016-06-07 23:59:59
2670196,3587071,2,2016-06-07 23:59:59,2016-06-08 00:03:45,3,0.34,-74.006569,40.721008,1,N,...,40.718071,1,4.0,0.5,0.5,1.59,0.00,0.3,6.890000,2016-06-07 23:59:59
2670197,3587072,2,2016-06-07 23:59:59,2016-06-08 00:20:04,1,7.04,-73.847450,40.754066,1,N,...,40.774597,2,23.0,0.5,0.5,0.00,0.00,0.3,24.299999,2016-06-07 23:59:59
2670198,3587073,2,2016-06-07 23:59:59,2016-06-08 00:13:50,1,6.00,-73.941093,40.839314,1,N,...,40.911297,1,18.0,0.5,0.5,2.16,2.54,0.3,24.000000,2016-06-07 23:59:59


# Citi Bike Dataframes
Since Citi Bike does not directly provide csv files, we must handle the zip files, extract, and load them accordingly.

In [0]:
reg_pattern = re.compile("^\d+.*\.csv$")

for n in YEARS:
  # See if the file exists
  file_path = get_csv_name("bike", n)
  if os.path.isfile(file_path) is True:
    continue

  # If not, request and save it
  req_url = ""
  if n != "2017" and n != "2018":
    req_url = "https://s3.amazonaws.com/tripdata/" + n + "06-citibike-tripdata.zip"
  else:
    req_url = "https://s3.amazonaws.com/tripdata/" + n + "06-citibike-tripdata.csv.zip"
    
  # Handle the ZIP
  z = zipfile.ZipFile(io.BytesIO(requests.get(req_url).content))
  orig_file_name = list(filter(reg_pattern.match, z.namelist()))[0]
  z.extract(orig_file_name)
  os.rename(orig_file_name, get_csv_name("bike", n)) # Renaming the file will automatically remove the original, extracted file

In [0]:
# The dtypes can't be passed in atm because some columns that should be integers have non-number strings in them
# Some columns that should be integers are floats instead because pandas can't put NaN values in integer columns
"""
dtypes = {
    "tripduration": np.int32,
    "starttime": "str",
    "stoptime": "str",
    "start station id": np.float32,
    "start station name": "str",
    "start station latitude": np.float32,
    "start station longitude": np.float32,
    "end station id": np.float32,
    "end station name": "str",
    "end station latitude": np.float32,
    "end station longitude": np.float32,
    "bikeid": np.int32,
    "usertype": "str",
    "birth year": np.float32,
    "gender": np.int32
}
"""


df_bike_2013_orig = pd.read_csv(get_csv_name("bike", "2013"))
df_bike_2014_orig = pd.read_csv(get_csv_name("bike", "2014"))
df_bike_2015_orig = pd.read_csv(get_csv_name("bike", "2015"))
df_bike_2016_orig = pd.read_csv(get_csv_name("bike", "2016"))
df_bike_2017_orig = pd.read_csv(get_csv_name("bike", "2017"))
df_bike_2018_orig = pd.read_csv(get_csv_name("bike", "2018"))

In [0]:
# Convert to datetime. This was really annoying with the different formats - if the format is unspecified, the functions never stop running.
# Also must assert that the last row of data has a start time greater than the second week, to make sure we have the full first 2 weeks of data

# 2013
df_bike_2013_orig['starttime_parsed'] = pd.to_datetime(df_bike_2013_orig['starttime'])
df_bike_2013_orig['stoptime_parsed'] = pd.to_datetime(df_bike_2013_orig['stoptime'])
assert df_bike_2013_orig.iloc[-1].starttime_parsed > datetime.strptime('2013-06-07', '%Y-%m-%d'), "Missing full 1 week of data for 2013"

# 2014
df_bike_2014_orig['starttime_parsed'] = pd.to_datetime(df_bike_2014_orig['starttime'])
df_bike_2014_orig['stoptime_parsed'] = pd.to_datetime(df_bike_2014_orig['stoptime'])
assert df_bike_2014_orig.iloc[-1].starttime_parsed > datetime.strptime('2014-06-07', '%Y-%m-%d'), "Missing full 1 week of data for 2014"

# 2015
df_bike_2015_orig['starttime_parsed'] = pd.to_datetime(df_bike_2015_orig["starttime"], format="%m/%d/%Y %H:%M")
df_bike_2015_orig['stoptime_parsed'] = pd.to_datetime(df_bike_2015_orig['stoptime'], format="%m/%d/%Y %H:%M")
assert df_bike_2015_orig.iloc[-1].starttime_parsed > datetime.strptime('2015-06-07', '%Y-%m-%d'), "Missing full 1 week of data for 2015"

# 2016
df_bike_2016_orig['starttime_parsed'] = pd.to_datetime(df_bike_2016_orig['starttime'], format="%m/%d/%Y %H:%M:%S")
df_bike_2016_orig['stoptime_parsed'] = pd.to_datetime(df_bike_2016_orig['stoptime'], format="%m/%d/%Y %H:%M:%S")
assert df_bike_2016_orig.iloc[-1].starttime_parsed > datetime.strptime('2016-06-07', '%Y-%m-%d'), "Missing full 1 week of data for 2016"

# 2017
df_bike_2017_orig['starttime_parsed'] = pd.to_datetime(df_bike_2017_orig['starttime'], format="%Y-%m-%d %H:%M:%S")
df_bike_2017_orig['stoptime_parsed'] = pd.to_datetime(df_bike_2017_orig['stoptime'], format="%Y-%m-%d %H:%M:%S")
assert df_bike_2017_orig.iloc[-1].starttime_parsed > datetime.strptime('2017-06-07', '%Y-%m-%d'), "Missing full 1 week of data for 2017"

# 2018
df_bike_2018_orig['starttime_parsed'] = pd.to_datetime(df_bike_2018_orig['starttime'], format="%Y-%m-%d %H:%M:%S")
df_bike_2018_orig['stoptime_parsed'] = pd.to_datetime(df_bike_2018_orig['stoptime'], format="%Y-%m-%d %H:%M:%S")
assert df_bike_2018_orig.iloc[-1].starttime_parsed > datetime.strptime('2018-06-07', '%Y-%m-%d'), "Missing full 1 week of data for 2018"

In [0]:
# Our main Citi Bike dataframes we'll be using, with a full week of data for each year

df_bike_2013 = df_bike_2013_orig[(df_bike_2013_orig['starttime_parsed'] > '2013-06-01') & (df_bike_2013_orig['starttime_parsed'] < '2013-06-08')]
df_bike_2014 = df_bike_2014_orig[(df_bike_2014_orig['starttime_parsed'] > '2014-06-01') & (df_bike_2014_orig['starttime_parsed'] < '2014-06-08')]
df_bike_2015 = df_bike_2015_orig[(df_bike_2015_orig['starttime_parsed'] > '2015-06-01') & (df_bike_2015_orig['starttime_parsed'] < '2015-06-08')]
df_bike_2016 = df_bike_2016_orig[(df_bike_2016_orig['starttime_parsed'] > '2016-06-01') & (df_bike_2016_orig['starttime_parsed'] < '2016-06-08')]
df_bike_2017 = df_bike_2017_orig[(df_bike_2017_orig['starttime_parsed'] > '2017-06-01') & (df_bike_2017_orig['starttime_parsed'] < '2017-06-08')]
df_bike_2018 = df_bike_2018_orig[(df_bike_2018_orig['starttime_parsed'] > '2018-06-01') & (df_bike_2018_orig['starttime_parsed'] < '2018-06-08')]


In [16]:
df_bike_2016.tail()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,starttime_parsed,stoptime_parsed
560365,316,6/7/2016 17:53:46,6/7/2016 17:59:02,2021,W 45 St & 8 Ave,40.759291,-73.988597,500,Broadway & W 51 St,40.762288,-73.983362,21298,Subscriber,1981.0,1,2016-06-07 17:53:46,2016-06-07 17:59:02
560366,2116,6/7/2016 17:53:52,6/7/2016 18:29:08,2021,W 45 St & 8 Ave,40.759291,-73.988597,147,Greenwich St & Warren St,40.715422,-74.011220,22022,Subscriber,1966.0,1,2016-06-07 17:53:52,2016-06-07 18:29:08
560367,1391,6/6/2016 15:45:15,6/6/2016 16:08:26,514,12 Ave & W 40 St,40.760875,-74.002777,327,Vesey Pl & River Terrace,40.715338,-74.016584,23948,Customer,NaN,0,2016-06-06 15:45:15,2016-06-06 16:08:26
560989,903,6/6/2016 17:25:18,6/6/2016 17:40:21,285,Broadway & E 14 St,40.734546,-73.990741,265,Stanton St & Chrystie St,40.722293,-73.991475,19904,Subscriber,1982.0,2,2016-06-06 17:25:18,2016-06-06 17:40:21
561924,642,6/6/2016 15:58:15,6/6/2016 16:08:57,282,Kent Ave & S 11 St,40.707645,-73.968415,539,Metropolitan Ave & Bedford Ave,40.715348,-73.960241,14791,Subscriber,1978.0,2,2016-06-06 15:58:15,2016-06-06 16:08:57
